In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


import gzip
import os
import gc

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

from keras.models import Sequential, load_model
from keras.layers import Conv1D, GlobalMaxPool1D, Dense, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
file_path1 = r"C:\Users\vicky\Documents\python\DS_ML\ds_priya\test.csv"
file_path2 = r"C:\Users\vicky\Documents\python\DS_ML\ds_priya\train.csv"

In [3]:
test = pd.read_csv(file_path1)
train = pd.read_csv(file_path2)

In [4]:
import pandas as pd
import numpy as np
import re
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



# Text cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\W", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.strip(' ')
    return text

# Apply cleaning to each comment
train['comment_text'] = train['comment_text'].apply(clean_text)

# Tokenization and padding
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 200
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train['comment_text'])
sequences = tokenizer.texts_to_sequences(train['comment_text'])
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Prepare labels
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y = train[labels].values

# Build the CNN model
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = Embedding(MAX_NB_WORDS, 128)(input_layer)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(6, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Predict probabilities
predictions = model.predict(X)

# Create submission DataFrame
submission = pd.DataFrame({'id': train['id']})
submission[labels] = predictions

# Save to CSV
submission.to_csv('submission.csv', index=False)


Epoch 1/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 250s 50ms/step - loss: 0.0940
Epoch 2/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 354s 71ms/step - loss: 0.0536
Epoch 3/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 264s 47ms/step - loss: 0.0488
Epoch 4/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 241s 48ms/step - loss: 0.0456
Epoch 5/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 199s 40ms/step - loss: 0.0419
Epoch 6/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 193s 39ms/step - loss: 0.0399
Epoch 7/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 193s 39ms/step - loss: 0.0375
Epoch 8/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 193s 39ms/step - loss: 0.0359
Epoch 9/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 198s 40ms/step - loss: 0.0337
Epoch 10/10
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 200s 40ms/step - loss: 0.0331
4987/4987 ━━━━━━━━━━━━━━━━━━━━ 46s 9ms/step
